In [33]:
import json
import os
import time
import re
import pandas as pd
import string


In [34]:

def twitter_feature_selection(json_file):
    selected_feature = {}
    selected_feature["text"] = json_file["text"]
    # selected_feature["id"] = json_file["id"]
    selected_feature["created_at"] = json_file["created_at"]
    # selected_feature['like_count'] = json_file['public_metrics']['like_count']
    selected_feature['retweet_count'] = json_file['public_metrics']['retweet_count']

    # selected_feature["hashtag"] = re.search(r'(\w+)+', json_file["text"])
    return selected_feature

def test_feature_selection(json_file):
    selected_feature = {}
    selected_feature["text"] = json_file["text"]
    selected_feature["created_at"] = json_file["created_at"]
    selected_feature['retweet_count'] = json_file['retweet_count']

    # selected_feature["id"] = json_file["id"]
    return selected_feature

def convert_label(label):
    if label == "rumour":
        return 1
    elif label == "nonrumour":
        return 0 
def merge_dictionary_list(dict_list):
      return {
    k: [d.get(k) for d in dict_list if k in d]
    for k in set().union(*dict_list)
  } 
def remove_unwanted_keys(dict_list, keys):
    for item in dict_list:
        item.pop(keys, None)
    return dict_list

In [35]:
def load_data(file_name):
    if (file_name == 'train' or file_name == 'dev'):
        total_json = []
        finished_dict = []
        with open("/Users/yuweng/Desktop/task1/" + file_name +".data.txt", "r") as f:
            file_name_event = f.readlines()      
        with open("/Users/yuweng/Desktop/task1/" + file_name +".label.txt", "r") as f:
            file_name_label = f.readlines()
        for event_str, event_label in zip(file_name_event, file_name_label):
            event_list = event_str.rstrip().split(",")
            event_label = event_label.rstrip().split(",")
            event_json = []
            path = "/Users/yuweng/Desktop/task1/" + file_name +'/' + event_list[0] + ".json"
            # print(path)
            # print(os.path.exists(path))
            if os.path.exists(path):
                for id in event_list:
                    path = "/Users/yuweng/Desktop/task1/" + file_name +'/' + id + ".json"
                    # print(path)
                    # print(os.path.exists(path))
                    if os.path.exists(path):
                        related_tweet = json.load(open(path, "r"))
                        # print(related_tweet)
                        event_json.append(twitter_feature_selection(related_tweet))
                        # print(event_json)
                event_json = sorted(event_json, key=lambda x: time.mktime(time.strptime(x["created_at"], '%Y-%m-%dT%H:%M:%S.%fZ')))
                event_json.append({"label":convert_label(''.join(event_label))})
                # print(event_json)
                merged_dict = merge_dictionary_list(event_json)
                # print(merged_dict)
                total_json.append(merged_dict)
                # print(total_json)
                finished_dict = remove_unwanted_keys(total_json, "created_at")
    elif (file_name == 'test'):
        total_json = []
        finished_dict = []
        with open("/Users/yuweng/Desktop/task1/" + file_name +".data.txt", "r") as f:
            file_name_event = f.readlines()      
        for event_str in file_name_event:
            event_list = event_str.rstrip().split(",")
            event_json = []
            path = "/Users/yuweng/Desktop/task1/" + file_name +'/' + event_list[0] + ".json"
            # print(path)
            # print(os.path.exists(path))
            if os.path.exists(path):
                for id in event_list:
                    path = "/Users/yuweng/Desktop/task1/" + file_name +'/' + id + ".json"
                    # print(path)
                    # print(os.path.exists(path))
                    if os.path.exists(path):
                        related_tweet = json.load(open(path, "r"))
                        # print(related_tweet)
                        event_json.append(test_feature_selection(related_tweet))
                event_json = sorted(event_json, key=lambda x: time.mktime(time.strptime(x["created_at"], '%a %b %d %H:%M:%S %z %Y')))
                # print(event_json)
                merged_dict = merge_dictionary_list(event_json)
                # print(merged_dict)
                total_json.append(merged_dict)
                # print(total_json)
                finished_dict = remove_unwanted_keys(total_json, "created_at")
    return pd.DataFrame(finished_dict)

Load the Required data set

In [36]:
df_train = load_data("train")
df_dev = load_data("dev")
df_test = load_data("test")

In [37]:
def text_preprocessing(tweet):
    r1 = r'https?:/\/\S+'
    ## remove @people
    r2 = r'@[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…？“”‘’！\\]+'
    ##remove tag
    r3 = r'#[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…？“”‘’！\\]+'

    tweet = tweet.encode('utf-8', 'replace').decode('utf-8')
    tweet = re.sub(r1, "", tweet)
    tweet = re.sub(r2, "", tweet)
    tweet = re.sub(r3, "", str(tweet).lower().strip())
    return tweet
def give_emoji_free_text(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF" 
        u"\U0001F1E0-\U0001F1FF"
        u"\U0001F1F2-\U0001F1F4"  
        u"\U0001F1E6-\U0001F1FF" 
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)
def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)
def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table).strip()


Join the Text together and do the data preprocessing

In [38]:
df_dev['text'][21]

['Can regularly rinsing your nose with saline help prevent infection with the new coronavirus? #COVID19 #21daysLockdownSA https://t.co/amR7IPw2C6',
 "@GP_CommSafety Hold on a bit and tell us why this virus after there was a movie of it? If you knew it was coming true then why didn't you find it's cure before now, you think lockdown will help? And how????? https://t.co/6ysbe4qFE6",
 '@GP_CommSafety Remove this post please']

In [39]:
import preprocessor as p

In [40]:
# df_train['text'] = df_train.apply(lambda x:text_preprocessing(''.join(x["text"])),axis=1)
# df_train['text'] = df_train['text'].apply(lambda x:x.lower().replace('\n',''))
# df_train['text'] = df_train['text'].apply(lambda x:give_emoji_free_text(x))
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION,p.OPT.RESERVED,p.OPT.EMOJI,p.OPT.SMILEY,p.OPT.NUMBER)
df_train.text = df_train.text.apply(lambda x : ''.join(x))
df_train.text = df_train.text.apply(lambda x : p.clean(x))
df_train['text'] = df_train['text'].apply(lambda x:x.lower().replace('\n','').replace("#", ""))

df_train['label'] = df_train.apply(lambda x:x["label"][0],axis=1)
df_train['label'] = df_train['label'].astype('int')
df_train['retweet_count'] = df_train.apply(lambda x:sum(x["retweet_count"]),axis=1)

# df_dev['text'] = df_dev.apply(lambda x:text_preprocessing(''.join(x["text"])),axis=1)

# df_dev['text'] = df_dev['text'].apply(lambda x:give_emoji_free_text(x))
df_dev.text = df_dev.text.apply(lambda x : ''.join(x))
df_dev.text = df_dev.text.apply(lambda x : p.clean(x))
df_dev['text'] = df_dev['text'].apply(lambda x:x.lower().replace('\n','').replace("#", ""))
df_dev['label'] = df_dev.apply(lambda x:x["label"][0],axis=1)
df_dev['label'] = df_dev['label'].astype('int')
df_dev['retweet_count'] = df_dev.apply(lambda x:sum(x["retweet_count"]),axis=1)

# df_test['text'] = df_test.apply(lambda x:text_preprocessing(''.join(x["text"])),axis=1)
# df_test['text'] = df_test['text'].apply(lambda x:x.lower().replace('\n',''))
# df_test['text'] = df_test['text'].apply(lambda x:give_emoji_free_text(x))
df_test.text = df_test.text.apply(lambda x : ''.join(x))
# p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.NUMBER)
df_test.text = df_test.text.apply(lambda x : p.clean(x))
df_test['text'] = df_test['text'].apply(lambda x:x.lower().replace('\n','').replace("#", ""))
df_test['retweet_count'] = df_test.apply(lambda x:sum(x["retweet_count"]),axis=1)







# df_train['text'] = df_train['text'].apply(lambda x:remove_URL(x))

# df_train['text'] = df_train['text'].apply(lambda x:remove_html(x))
# df_train['text'] = df_train['text'].apply(lambda x:remove_punct(x))

# df_dev['text'] = df_dev['text'].apply(lambda x:remove_URL(x))

# df_dev['text'] = df_dev['text'].apply(lambda x:remove_html(x))
# df_dev['text'] = df_dev['text'].apply(lambda x:remove_punct(x))

# df_test['text'] = df_test['text'].apply(lambda x:remove_URL(x))

# df_test['text'] = df_test['text'].apply(lambda x:remove_html(x))
# df_test['text'] = df_test['text'].apply(lambda x:remove_punct(x))

# df_train = df_train.dropna(axis=0, how='any')
# df_dev = df_dev.dropna(axis=0, how='any')
# df_test = df_test.dropna(axis=0, how='any')

In [41]:
df_dev['text'][15]

'google south american goliath birdeater. helllll noooooo they just keep finding stuff not fallin for that again nasty!!!!!! no reason it should even exist ok nooooo wtf a puppy sized spiderthis is the spider : google south american goliath birdeater. helllll noooooo no no no no no no! that\'s a horror movie spider! lol! i\'m the same way with rats.... i thought it was going to be a bird byeeeee : google south american goliath birdeater. helllll noooooo i just cringed!!!": google south american goliath birdeater. helllll noooooo" why would you do this to me? hence why i\'m arachnophobic!!!! .. : google south american goliath birdeater. helllll noooooo gurllll. i got one for you. google camel spider. omg ah i hate you... my curiosity got the best of me hahahaha fuck u why would u make me: google south american goliath birdeater. helllll noooooo: google south american goliath birdeater. helllll noooooo omg i noped that so hard! i threw my phone across the room fuck you omg lol holy balls

In [42]:
df_test

,text,retweet_count
0,"how does covid-19 spread? thanks, wcco! you ar...",0
1,"if you dont believe me, all you have to do is ...",0
2,q. how are covid-19 and influenza viruses simi...,0
3,una de les q&amp;a on coronaviruses de la pgin...,5
4,we cant forget that racism is in all our insti...,6
...,...,...
553,ex-marlboro man dies from smoking-related dise...,140
554,holy shit. doritos flavored mountain dew. all ...,182
555,banksy account joins cartoonists support for c...,351
556,q: who are the members and advisers of the int...,69


In [43]:
df_train

,label,text,retweet_count
0,0,. can eating garlic help prevent infection wit...,0
1,1,french police chief killed himself after charl...,185
2,0,coronavirus disease (covid-19) advice for the ...,1
3,0,ottawa police confirm that there were multiple...,123
4,0,if the primary focus of a government isn't to ...,1
...,...,...,...
1558,0,. it cannot be transmitted through goods manuf...,1
1559,1,desperate ted cruz claims planned parenthood s...,83
1560,1,"""thoughts and prayers are not enough."" pres. o...",109
1561,0,police have surrounded this building where the...,177


In [44]:
df_train.to_csv('train.csv', index=False)
df_dev.to_csv('dev.csv', index = False)
df_test.to_csv('test.csv', index = False)

In [45]:
df_train = pd.read_csv('/Users/yuweng/Desktop/task1/train.csv')
df_dev = pd.read_csv('/Users/yuweng/Desktop/task1/dev.csv')
df_test = pd.read_csv('/Users/yuweng/Desktop/task1/test.csv')

In [46]:
df_train.isnull().any()

label            False
text             False
retweet_count    False
dtype: bool

In [47]:
df_test.isnull().any()

text              True
retweet_count    False
dtype: bool

In [48]:
df_train.dropna(axis=0, how='any', inplace=True)
df_dev.dropna(axis=0, how='any', inplace=True)
# df_test.dropna(axis=0, how='any', inplace=True)

In [49]:
is_NaN = df_test.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df_test[row_has_NaN]


In [50]:
row_has_NaN

0      False
1      False
2      False
3      False
4      False
       ...  
553    False
554    False
555    False
556    False
557    False
Length: 558, dtype: bool

In [51]:
df_train.to_csv("train.csv",index= False)
df_dev.to_csv('dev.csv',index= False)
df_test.to_csv('test.csv',index= False)

In [52]:
df_test

,text,retweet_count
0,"how does covid-19 spread? thanks, wcco! you ar...",0
1,"if you dont believe me, all you have to do is ...",0
2,q. how are covid-19 and influenza viruses simi...,0
3,una de les q&amp;a on coronaviruses de la pgin...,5
4,we cant forget that racism is in all our insti...,6
...,...,...
553,ex-marlboro man dies from smoking-related dise...,140
554,holy shit. doritos flavored mountain dew. all ...,182
555,banksy account joins cartoonists support for c...,351
556,q: who are the members and advisers of the int...,69
